In [9]:
import torch
import torchvision.models as models 
import torchvision.transforms as transforms
from PIL import Image

In [10]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

cuda


In [11]:
model = models.resnet18(pretrained = True) #开启预训练，迁移参数
model = torch.nn.Sequential(*list(model.children())[:-1]) #最后一层不要
model = model.to(device)
model.eval()

/home/crisi/miniconda3/envs/d2l/lib/python3.9/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/crisi/miniconda3/envs/d2l/lib/python3.9/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Sequential(
  (0): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (2): ReLU(inplace=True)
  (3): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (4): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Con

In [12]:
#预处理图，对原图像做预处理中同样的操作
def pre_process_image(img_path):
    image = Image.open(img_path)
    preprecess =  transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229,0.224,0.225])
    ])
    #添加batch维度，移动到GPU上
    return preprecess(image).unsqueeze(0).to(device)

In [26]:
target = 3
data_name = ['0618', '0854', '1066'][target-1]

image_path = f'../input_data/{data_name}.png'
image_tensor = pre_process_image(image_path)

with torch.no_grad():
    features = model(image_tensor)

print(features.shape)
print(type(features))

torch.Size([1, 512, 1, 1])
<class 'torch.Tensor'>


In [27]:
import numpy as np
features_up = features.cpu().numpy()
save_path = f'./Features/{data_name}_NN_features.npy'
np.save(save_path, features_up)
print(f'Features saved to {save_path}')

Features saved to ./Features/1066_NN_features.npy
